Expoiting the John Hopkins data related to covid-19 deaths and confermed new cases I tried to implement a basic but easy to read world plot in which is it possible to specify the desired day. 

In [ ]:
import geopandas as gpd
import numpy as np 
import pandas as pd 
import datetime as dt
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.figure_factory as ff

In [ ]:
def covid_plot(df, abs = True, Today=True, date=None):
    '''
    World plot of covid-19 confirmed cases and death. 
    df = string, need to be 'confirmed_cases' or 'deaths'
    abs= Boolean, default is True. Does the data must be reported in absolute or relative value? Relative value is calculated based on the  country population devided by one milion.
    Today= Boolean, default is True. It is a parameter to specify if the data, are the today data, if set equal to false, a specific date must be specified
    date= dt.datetime, in case Today=False, it is possible to specify a specific date for the plot. It needs to be in dt.datetime format, so be sure 
          to inser date = dt.datetime(YYYY,M,DD) format. The data starts from 2020,1,23, so be sure to select a date after this one.
    '''
    
    if df != 'deaths' and df != 'confirmed_cases':
        print('available option for df are: "deaths" or "confirmed_cases"')
    if df == 'deaths' or df == 'confirmed_cases':
        initial_data=dt.datetime(2020,1,23)

        us = pd.read_csv('../input/covid19-data-from-john-hopkins-university/CONVENIENT_us_{}.csv'.format(df),low_memory=False).iloc[1:,1:]
        us = np.sum(us.astype(float),axis=1)
        us = us.rename('United States of America')
        
        data = dt.date.today().strftime("%d/%m/%Y")

        death = pd.read_csv('../input/covid19-data-from-john-hopkins-university/CONVENIENT_global_{}.csv'.format(df))
        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
        
        df_new=pd.DataFrame()
        states=['Australia','Canada','China','Denmark','France','Netherlands','United Kingdom']
        
        for i in states:
            colonna = ([col for col in death.columns if i in col])
            df_new =  pd.concat([df_new,(np.sum(death[colonna].iloc[1:].astype(float),axis=1))],axis=1)
            death = death.drop(columns=colonna)
        df_new.columns = states
    
        death = death.drop(columns=['Country/Region'])
        death = pd.concat([death,df_new],axis=1)
        death = death.T 
        death = death.drop(columns=[0])

        death = death.append(us)
        state = death.index
        death.index = range(0,len(death))
        death['name']= state

        ########################
        
        if Today==True and date==None:
            test = pd.merge(world,death.iloc[:,-2:],on=['name'],how='left')
            test.iloc[:,-1] = test.iloc[:,-1].apply(lambda x: float(x))
            
            if abs==False:
                population = test.iloc[:,0]
                population_mil=population//1000000
                test.iloc[:,-1] = test.iloc[:,-1] //population_mil

        if Today==True and date!=None:
            print('Choose only a date!')

        if Today==False:
            if date==None:
                 print('Either specifying a date or select Today = True to set date for today.')
        
        if date!=None:
            if type(date) != dt.datetime:
                print('be sure that date format is: "dt.datetime(YYYY,M,DD)"')

            if type(date) == dt.datetime:
                data = date
                index=((data-initial_data).days)
                if index < 0:
                    print('data are recorded starting from 2020,1,23; be sure to select a value after this date')
                if index >= 0:
                    data = (data.strftime("%d/%m/%Y"))
                    test = pd.merge(world,death[[index,'name']],on=['name'],how='left')
                    test.iloc[:,-1] = test.iloc[:,-1].apply(lambda x: float(x))
                
            if abs==False:
                population = test.iloc[:,0]
                population_mil=population//1000000
                test.iloc[:,-1] = test.iloc[:,-1] //population_mil


        ab = 'Absolute'
        if abs==False:
            ab= 'Relative'
        
        ######################## Using pyplot
    ranges = [0,np.max(test.iloc[:,-1])]
    
    if df == 'confirmed_cases' and abs==True:
        ranges= [0,200000]
    if df == 'confirmed_cases' and abs==False:
        ranges = [0,1000]
    if df == 'deaths' and abs==False:
        ranges = [0,35]
    if df == 'deaths' and abs==True:
        ranges=[0,3000]
        
        
    fig = px.choropleth(test, locations=test.iloc[:,2],
                    color_continuous_scale="Viridis",
                    locationmode='country names',
                    color=test.iloc[:,-1].values,
                    #color_continuous_scale=px.colors.sequential.OrRd,
                    labels={'test.iloc[:,-1].values':'unemployment rate'},
                    width = 1050,
                    height= 600,
                    range_color=ranges
                       )
    fig.update_layout(title="{} world covid-19 {} in {}".format(ab,df,data))
    fig.show()
    
        ######################## Using geopandas
        
#         if  df ==  'confirmed_cases':
#             maxs=200000
#             if abs==False:
#                 maxs=1000
#         if  df == 'deaths':
#             maxs=3500
#             if abs==False:
#                 maxs=35        
        
#         fig,ax=plt.subplots(figsize=(25,25))
#         test.plot(column=test.iloc[:,-1],vmin=0,vmax=maxs,legend=True, 
#               legend_kwds={
#         'label': "number of {}".format(df),
#         'orientation': "horizontal"},
#               missing_kwds={
#                 'color':'lightgrey',
#                 'label':'Missing values'
#               } , ax=ax
#              )
#         ax.set_title("{} world covid-19 {} in {}".format(ab,df,data), fontsize=25)


In [ ]:
help(covid_plot)

In [ ]:
covid_plot('deaths')

In [ ]:
covid_plot('deaths',abs=False)

In [ ]:
covid_plot('confirmed_cases')

In [ ]:
covid_plot('confirmed_cases',abs=False)